In [1]:
pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 58.9 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [20]:
# To suppress all warnings

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from absl import logging
logging.set_verbosity(logging.ERROR)

import tensorflow as tf

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="keras")

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # hide TF logs

import tensorflow as tf
from tensorflow import keras

import os
os.environ['XLA_FLAGS'] = '--xla_cpu_enable_fast_math=false'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import warnings
from rasterio.errors import NotGeoreferencedWarning

warnings.filterwarnings("ignore", category=NotGeoreferencedWarning)




In [2]:
import os
import numpy as np
import rasterio
from skimage.transform import resize
import matplotlib.pyplot as plt

DATA_DIR = "/kaggle/input/satellite-images"
TRAIN_DIR = ("/kaggle/input/dataset11/train")
TEST_DIR = ("/kaggle/input/dataset11/test")

In [3]:
import os

print("Train files:", os.listdir(TRAIN_DIR))
print("Test files:", os.listdir(TEST_DIR))


Train files: ['2_image.tif', '42_image.tif', '18_image.tif', '35_label.tif', '64_label.tif', '62_label.tif', '8_image.tif', '32_image.tif', '27_image.tif', '22_image.tif', '2_label.tif', '68_label.tif', '28_image.tif', '67_image.tif', '47_label.tif', '26_label.tif', '1_label.tif', '8_label.tif', '63_label.tif', '17_label.tif', '45_label.tif', '52_label.tif', '44_image.tif', '33_image.tif', '20_label.tif', '46_label.tif', '25_label.tif', '49_label.tif', '43_image.tif', '26_image.tif', '22_label.tif', '24_label.tif', '58_image.tif', '34_image.tif', '48_label.tif', '53_image.tif', '28_label.tif', '30_label.tif', '34_label.tif', '65_label.tif', '59_image.tif', '45_image.tif', '31_image.tif', '67_label.tif', '20_image.tif', '64_image.tif', '17_image.tif', '42_label.tif', '0_image.tif', '62_image.tif', '52_image.tif', '0_label.tif', '12_label.tif', '29_label.tif', '33_label.tif', '69_image.tif', '14_label.tif', '39_image.tif', '5_image.tif', '9_image.tif', '7_label.tif', '57_label.tif', '21_

In [21]:
# Path to an image
image_path = os.path.join(TRAIN_DIR, "0_image.tif")

# Open the image and inspect its metadata
with rasterio.open(image_path) as src:
    print("Image Metadata:")
    print(src.meta)
    print(f"Number of Bands: {src.count}")
    print(f"Data Type: {src.dtypes}")
    print(f"Shape: {src.height} x {src.width}")
    print(f"CRS: {src.crs}")


mask_path = os.path.join(TRAIN_DIR, "0_label.tif")
with rasterio.open(mask_path) as src:
    print("\nMask Metadata:")
    print(src.meta)
    print(f"Number of Bands: {src.count}")
    print(f"Data Type: {src.dtypes}")
    print(f"Shape: {src.height} x {src.width}")
    print(f"CRS: {src.crs}")

Image Metadata:
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 128, 'height': 128, 'count': 3, 'crs': None, 'transform': Affine(1.0, 0.0, 0.0,
       0.0, 1.0, 0.0)}
Number of Bands: 3
Data Type: ('uint8', 'uint8', 'uint8')
Shape: 128 x 128
CRS: None

Mask Metadata:
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 128, 'height': 128, 'count': 1, 'crs': None, 'transform': Affine(1.0, 0.0, 0.0,
       0.0, 1.0, 0.0)}
Number of Bands: 1
Data Type: ('float32',)
Shape: 128 x 128
CRS: None


In [ ]:
def load_data(image_path, mask_path):
    # Load image
    with rasterio.open(image_path) as src:
        image = src.read()  # Read all bands
        image = np.transpose(image, (1, 2, 0))  # Reorder dimensions to (height, width, channels)


        if image.shape[2] == 4:  
            image = image[..., :3]

    # Load mask
    with rasterio.open(mask_path) as src:
        mask = src.read(1)  # Read single band (binary mask)

    return image, mask

In [6]:
print(f"Train Images: {len(train_image_paths)}")
print(f"Train Masks: {len(train_mask_paths)}")
print(f"Test Images: {len(test_image_paths)}")
print(f"Test Masks: {len(test_mask_paths)}")

Train Images: 70
Train Masks: 70
Test Images: 30
Test Masks: 30
